In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# I have a cnn model that I trained on the CIFAR10 dataset.
# I want to see how robust it is to adversarial attacks.

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision

from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from ml_security.adaptative_network.eval.utils import PreActResNet, PreActResNetwithKAN, PreActBlock
from tqdm import tqdm



from ml_security.datasets.datasets import (
    DATASET_REGISTRY,
    DatasetType,
    create_dataloader,
)
from ml_security.logger import logger
from ml_security.utils import get_device, set_seed


model_path = "cnn/CIFAR10/classic_cnn.pth"

# Load the model
model = PreActResNet(PreActBlock, [2, 2, 2, 2])
model.load_state_dict(torch.load(model_path))
model.eval()



PreActResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (layer1): Sequential(
    (0): PreActBlock(
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (1): PreActBlock(
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
  )
  (layer2): Sequential(
    (0): PreActBlock(
      (bn1): BatchNorm2d

In [5]:
dataset = "CIFAR10"

dataset = DatasetType[dataset]
dataset_info = DATASET_REGISTRY[dataset]

transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
    ]
)
if dataset_info.origin == "TORCHVISION":
    trainloader = create_dataloader(
        dataset=dataset, batch_size=64, train=True, transformation=transform
    )
    valloader = create_dataloader(
        dataset=dataset, batch_size=64, train=False, transformation=transform
    )
else:
    raise ValueError("Unknown dataset origin.")

100%|██████████| 170498071/170498071 [00:06<00:00, 24653975.79it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
